<a href="https://colab.research.google.com/github/lko9911/Artificial-Intelligence-Project/blob/main/YOLO/yolo_seg(%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ultralytics import YOLO
import os
import cv2
import numpy as np
from glob import glob

# ⚙️ 데이터셋 yaml 정의 (64개 클래스로 업데이트됨)
data_yaml = """\
# ------------------------------------------------------------------
# YOLOv11 (Ultralytics) 데이터셋 구성 파일
# ------------------------------------------------------------------
path: /content/drive/MyDrive/Dataset_NYU
train: train/images
val: val/images

# ------------------------------------------------------------------
# 클래스 정의 (총 64개)
# 이 순서는 TXT 파일의 숫자 ID (0부터 63)와 정확히 일치합니다.
# ------------------------------------------------------------------
nc: 64
names:
  0: airduct
  1: airvent
  2: bag
  3: ball
  4: bar
  5: basket
  6: book
  7: bottle
  8: bowl
  9: box
  10: cabinet
  11: camera
  12: ceiling
  13: chair
  14: clock
  15: computer
  16: cone
  17: corkboard
  18: counter
  19: cup
  20: desk
  21: dishwasher
  22: door
  23: doorknob
  24: faucet
  25: fireextinguisher
  26: floor
  27: garbagebin
  28: greenscreen
  29: holepuncher
  30: keyboard
  31: ladder
  32: laptop
  33: light
  34: magnet
  35: manillaenvelope
  36: mantel
  37: microwave
  38: monitor
  39: motioncamera
  40: paper
  41: papertoweldispenser
  42: picture
  43: pipe
  44: pot
  45: projectorscreen
  46: refridgerator
  47: scissor
  48: shelves
  49: sink
  50: speaker
  51: stackedchairs
  52: stand
  53: stoveburner
  54: styrofoamobject
  55: table
  56: tapedispenser
  57: telephone
  58: telephonecord
  59: tracklight
  60: unknown
  61: wall
  62: whiteboard
  63: window
"""

yaml_path = "nyu_seg_data.yaml"
with open(yaml_path, 'w') as f:
    f.write(data_yaml)

print(f"'{yaml_path}' 파일이 64개 클래스 정보로 업데이트되었습니다.")
# 이후 학습 코드는 여기에 추가합니다 (예: model = YOLO('yolov8n-seg.pt'))

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
'nyu_seg_data.yaml' 파일이 64개 클래스 정보로 업데이트되었습니다.


In [ ]:
import os
import shutil
from glob import glob
import random
from tqdm import tqdm  # tqdm 추가

DATASET_ROOT = '/content/drive/MyDrive/Dataset_NYU'
SRC_IMAGES = os.path.join(DATASET_ROOT, 'images')
SRC_LABELS = os.path.join(DATASET_ROOT, 'labels')

SPLITS = ['train', 'val', 'test']
for split in SPLITS:
    os.makedirs(os.path.join(DATASET_ROOT, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(DATASET_ROOT, split, 'labels'), exist_ok=True)

# 모든 이미지 확장자 포함
image_paths = sorted(glob(os.path.join(SRC_IMAGES, '*.*')))  # png, jpg, jpeg 등
random.shuffle(image_paths)

num_total = len(image_paths)
num_train = int(0.7 * num_total)
num_val   = int(0.2 * num_total)
num_test  = num_total - num_train - num_val

splits = {
    'train': image_paths[:num_train],
    'val': image_paths[num_train:num_train+num_val],
    'test': image_paths[num_train+num_val:]
}

for split, paths in splits.items():
    print(f"\n[{split.upper()}] 진행 중... ({len(paths)}개)")
    for img_path in tqdm(paths):
        img_name = os.path.basename(img_path)
        name_wo_ext = os.path.splitext(img_name)[0]

        # 이미지 복사
        shutil.copy(img_path, os.path.join(DATASET_ROOT, split, 'images', img_name))

        # 라벨 복사
        label_path = os.path.join(SRC_LABELS, f"{name_wo_ext}.txt")
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(DATASET_ROOT, split, 'labels', f"{name_wo_ext}.txt"))

print("\n데이터셋 복사 완료!")


[TRAIN] 진행 중... (1014개)


100%|██████████| 1014/1014 [14:30<00:00,  1.16it/s]



[VAL] 진행 중... (289개)


100%|██████████| 289/289 [03:43<00:00,  1.29it/s]



[TEST] 진행 중... (146개)


100%|██████████| 146/146 [02:11<00:00,  1.11it/s]


데이터셋 복사 완료!


In [ ]:
from glob import glob
import os

split = 'train'
#mask_dir = f"/content/drive/MyDrive/Dataset_NYU/{split}/labels"
mask_dir = f"/content/drive/MyDrive/Dataset_NYU/{split}/images"
mask_paths = glob(f"{mask_dir}/*.jpg")
print(f"📂 디렉토리 경로: {mask_dir}")
print(f"🔍 마스크 파일 수: {len(mask_paths)}")

if len(mask_paths) > 0:
    print("📑 예시 파일:", os.path.basename(mask_paths[0]))
else:
    print("🚫 마스크 파일이 없음!")


📂 디렉토리 경로: /content/drive/MyDrive/Dataset_NYU/train/images
🔍 마스크 파일 수: 1014
📑 예시 파일: NYU0330.jpg


In [ ]:
# ⚙️ 모델 로드 및 학습
model = YOLO('yolo11n-seg.pt')  # pretrained model

model.train(
    data=yaml_path,
    epochs=100,
    imgsz=512,
    batch=32,
)

Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=nyu_seg_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, 

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 29, 30, 31, 32, 33, 35, 36, 37, 38, 40, 42, 43, 44, 46, 47, 48, 49, 50, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x797a37fd2c60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0

In [ ]:
import os
import shutil
from google.colab import drive

# Google 드라이브 마운트
drive.mount('/content/drive')

# 원본 경로
source_path = '/content/runs'

# 목적지 경로
destination_path = '/content/drive/MyDrive/yolo_seg_project'

# 목적지 경로가 없으면 생성
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# source_path 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
